<a href="https://colab.research.google.com/github/harish-03-11/Google-word-2-vec-gradient-boosting/blob/main/Google_word2vec_Gradient_boost_fake_Vs_True.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gensim.downloader as api

In [2]:
wv = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [3]:
wv.doesnt_match(['dog', 'peigion', 'crow', 'bird'])

'dog'

In [4]:
wv.similarity(w1='dog', w2='cat')

0.76094574

In [5]:
wv.most_similar("microsoft")

[('adobe_photoshop', 0.8042365908622742),
 ('microsoft_office', 0.7978680729866028),
 ('windows_xp', 0.7926486134529114),
 ('buy_microsoft', 0.7902629375457764),
 ('cs4', 0.7494896650314331),
 ('autocad', 0.7432770729064941),
 ('photoshop', 0.7404437065124512),
 ('windows_vista', 0.7382056713104248),
 ('quickbooks', 0.7320875525474548),
 ('adobe_photoshop_cs4', 0.7269179821014404)]

In [6]:
glv = api.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [7]:
fruits = ["banana apple kiwi".split()]

In [8]:
fruits

[['banana', 'apple', 'kiwi']]

In [9]:
wv['good'].shape

(300,)

In [10]:
import pandas as pd

In [12]:
df_true = pd.read_csv("/content/sample_data/True.csv")
df_fake = pd.read_csv("/content/sample_data/Fake.csv")


In [13]:
import random

num_rows_to_drop_true = 19417
random_indices_true = random.sample(range(len(df_true)), num_rows_to_drop_true)
df_true = df_true.drop(random_indices_true, axis=0)
df_true.shape

(2000, 4)

In [14]:


num_rows_to_drop_fake = 21481
random_indices_fake = random.sample(range(len(df_fake)), num_rows_to_drop_fake)
df_fake = df_fake.drop(random_indices_fake, axis=0)
df_fake.shape

(2000, 4)

In [15]:
df_true['label'] = 1

In [16]:
df_fake['label'] = 0

In [17]:
df_combined = pd.concat([df_true,df_fake],axis=0)
df_combined.shape


(4000, 5)

In [18]:

df = df_combined.sample(frac=1)

In [19]:

df = df.drop(['title','subject','date'],axis = 1)

In [20]:
df.sample(3)

,text,label
3081,WASHINGTON (Reuters) - The White House is brin...,1
2040,(Reuters) - A federal court judge on Wednesday...,1
9822,"RICHMOND, Va. (Reuters) - Virginia Governor Te...",1


In [21]:
!python -m spacy download en_core_web_lg
import spacy
nlp = spacy.load("en_core_web_lg")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 626.2 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [22]:
def preprocess_vector(text):
  filtered = []
  doc = nlp(text)
  for token in doc:
    if not token.is_punct or token.is_stop:
      filtered.append(token.lemma_)
  return wv.get_mean_vector(filtered)



In [23]:
df.head(3)

,text,label
13572,SINGAPORE (Reuters) - Chinese President Xi Jin...,1
10277,"Sometimes, the left just makes it far too easy...",0
16908,WASHINGTON (Reuters) - The United States does ...,1


In [24]:
df['vectors'] = df['text'].apply(lambda text: preprocess_vector(text))

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X = df['vectors']
y = df['label']

In [27]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 42,stratify=y)

In [29]:
import numpy as np
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)


In [31]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report


In [32]:
model = GradientBoostingClassifier()

In [33]:
model.fit(X_train_2d,y_train)

GradientBoostingClassifier()

In [34]:
y_pred = model.predict(X_test_2d)

In [35]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94       401
           1       0.94      0.94      0.94       399

    accuracy                           0.94       800
   macro avg       0.94      0.94      0.94       800
weighted avg       0.94      0.94      0.94       800

